In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import seaborn as sb
import numpy as np

import geopandas as gpd

import folium

In [2]:
df = pd.read_csv('US_MaizeYield Predictors_1950-2013.csv')

In [3]:
df.index = pd.to_datetime(df.year, format='%Y')
df['state'] = df['state'].astype(int)
df_states =  pd.read_csv('states.csv', header=0, 
                         names=['name', 'zip', 'FIPS'])
st_dict = { int(row['FIPS']): row['zip'] for index, row in df_states.iterrows()}
df['state'] = df['state'].map(st_dict)

In [4]:
df

,yield,year,state,fips,area,EDD_4,GDD_4,prcp_4,SM_mean_4,SM_week_max_4,...,prcp2_8,EDD_9,GDD_9,prcp_9,SM_mean_9,SM_week_max_9,SM_week_min_9,SM_mean2_9,prcp2_9,maize_irr_frac
year,,,,,,,,,,,,,,,,,,,,,
2013-01-01,182.2,2013.0,AL,1033,25500.0,0.144660,182.989896,138.405888,30.587954,31.415827,...,16804.874644,12.602523,388.188638,96.585943,27.869618,29.376769,25.102011,781.745164,9666.466330,0.119278
2012-01-01,102.4,2012.0,AL,1033,24500.0,0.444557,226.509776,39.489620,26.918694,30.273795,...,6367.565199,6.820552,355.923101,152.952440,28.802978,30.817406,25.627490,834.572860,23509.203313,0.119278
2011-01-01,131.3,2011.0,AL,1033,19500.0,0.952782,244.578863,326.529743,31.492787,32.338234,...,17689.150161,7.688949,325.605600,150.150769,28.074765,30.903118,22.073456,792.904040,23026.178185,0.119278
2010-01-01,142.8,2010.0,AL,1033,20800.0,0.217440,224.846848,64.150614,26.646758,28.437905,...,10075.421901,24.915340,381.107820,67.905520,25.920732,26.574370,23.648074,676.753567,4650.376875,0.119278
2009-01-01,126.0,2009.0,AL,1033,16400.0,0.548458,186.166730,105.614995,29.474745,31.515805,...,4963.978537,1.771131,386.448506,246.002095,30.366801,34.609346,23.413031,927.362943,60992.173336,0.119278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1959-01-01,50.0,1959.0,WY,56031,3800.0,0.000000,67.360842,21.488786,15.913637,17.914976,...,279.442392,4.064808,162.035103,57.082121,16.413762,23.433538,11.073422,274.056968,3329.812711,1.000000
1958-01-01,45.0,1958.0,WY,56031,2700.0,0.000000,32.878001,39.902726,18.876361,21.586584,...,935.592267,3.528275,215.410403,5.367688,11.461390,12.145675,11.094129,134.913164,38.006369,1.000000
1957-01-01,42.0,1957.0,WY,56031,3200.0,0.000000,27.029601,57.170788,18.207538,21.572018,...,1836.174876,0.269799,148.678870,32.571971,18.034781,21.384815,13.703382,330.142497,1092.752807,1.000000


In [5]:
# We import the geoJSON file. 
url = ("https://raw.githubusercontent.com/python-visualization/folium/master/examples/data")
state_geo = f"{url}/us_counties_20m_topo.json"
# We read the file and print it.
geoJSON_df = gpd.read_file(state_geo)
geoJSON_df = geoJSON_df.rename(columns = {"id":"postal_code"})
geoJSON_df.head()

,postal_code,geometry
0,0500000US01001,"POLYGON ((-86.49932 32.34394, -86.71470 32.402..."
1,0500000US01009,"POLYGON ((-86.57112 33.76632, -86.75060 33.841..."
2,0500000US01017,"POLYGON ((-85.17117 32.86797, -85.13527 32.771..."
3,0500000US01021,"POLYGON ((-86.53522 33.02305, -86.49932 32.926..."
4,0500000US01033,"POLYGON ((-88.15055 34.57911, -88.15055 34.589..."


In [6]:
final_df = geoJSON_df.merge(df, on ="postal_code")
final_df.head(2)

KeyError: 'postal_code'

In [ ]:
from ipyleaflet import Map


In [ ]:
fmap = folium.Map(location=[38, -102], zoom_start=4)


folium.Choropleth(
                  geo_data=final_df,
                  data=final_df,
                  columns=['postal_code',"prcp_9"],
                  key_on="feature.properties.state",
                  fill_color='YlGnBu',
                  fill_opacity=1,
                  line_opacity=0.2,
                  legend_name="prcp_9",
                  smooth_factor=0,
                  Highlight= True,
                  line_color = "#0000",
                  name = "Wills",
                  show=False,
                  overlay=True,
                  nan_fill_color = "White"
                  ).add_to(fmap)



In [ ]:
fmap

In [ ]:
ds_corr = final_df.corr()

mask = np.triu(np.ones_like(ds_corr, dtype=bool))

fig, ax = plt.subplots(figsize=(15,15))

cmap = 'coolwarm'

sb.heatmap(ds_corr,mask=mask, annot=False, fmt='.2f', cmap=cmap, vmin=-1, vmax=1, square=True, linewidth=.7)

In [ ]:
ds.isna().sum()